In [3]:
!export LD_PRELOAD=/usr/lib/aarch64-linux-gnu/libgomp.so.1

In [7]:
import glob
!pwd

/home/sdpl/Sadaf/flask_app


In [4]:
# import tensorflow as tf
# import os
# facenet_model = tf.keras.models.load_model("84_percent_accuracy.tflite")
# tf.keras.models.save_model(facenet_model, 'New_Model.h5') # Saving the Model in H5 Format
# loaded_model_from_h5 = tf.keras.models.load_model('New_Model.h5') # Loading the H5 Saved Model
# print(loaded_model_from_h5.summary())

In [ ]:
#blotch
# For file file, the output is [[92 80 64 20]]
#normal
# For file file, the output is [[ 16 166  56  17]]
#rotton
# For file file, the output is [[ 46   2 140  68]]
#scab
# For file file, the output is [[ 28  11  87 129]]

# {0:"blotch",1:"normal",2:"rotten",3:"scab"}

In [28]:

import cv2 
import tensorflow as tf
import numpy as np


labels=["blotch","normal","rotten","scab"]
# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path="84_percent_accuracy.tflite")
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
print(input_details)
output_details = interpreter.get_output_details()

path = r'static/normal1.jpeg'
img = cv2.imread(path)
new_img = cv2.resize(img, (224,224))
print(new_img.shape)
new_img = np.array(new_img)
# new_img = np.transpose(new_img, (2, 0, 1))
# input_details[0]['index'] = the index which accepts the input
# expected type FLOAT32 for input 0, name: serving_default_input_1 

interpreter.set_tensor(input_details[0]['index'], [new_img])

# run the inference
interpreter.invoke()

# output_details[0]['index'] = the index which provides the input
output_data = interpreter.get_tensor(output_details[0]['index'])
minval=min(output_data[0])
maxval=max(output_data[0])
results=[(i-minval)/(minval+maxval) for i in output_data[0]]
print(round(results[2],2))
output_map=[ i for i in zip(labels,results)]

print("For file {}, the output is {}".format("file", output_map))
for label,val in output_map:
    print(label,val)

[{'name': 'input_6', 'index': 0, 'shape': array([  1, 224, 224,   3], dtype=int32), 'shape_signature': array([ -1, 224, 224,   3], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (0.003921568859368563, 0), 'quantization_parameters': {'scales': array([0.00392157], dtype=float32), 'zero_points': array([0], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
(224, 224, 3)
0.22
For file file, the output is [('blotch', 0.0), ('normal', 0.82417583), ('rotten', 0.21978022), ('scab', 0.0054945056)]
blotch 0.0
normal 0.82417583
rotten 0.21978022
scab 0.0054945056
